In [1]:
# импортируем необходимые библиотеки и модули
import pandas as pd
import numpy as np
import optuna
import random
from functools import partial
from pathlib import Path
from typing import Optional
import typer
import os

# импортируем из библиотеки etna нужные классы
from etna.datasets import TSDataset, generate_ar_df
from etna.loggers import WandbLogger,tslogger
from etna.metrics import MAE, MSE, SMAPE, Sign
from etna.models import CatBoostModelMultiSegment
from etna.pipeline import Pipeline
from etna.transforms import (LagTransform,
                             SegmentEncoderTransform,
                             StandardScalerTransform)

# импортируем модуль предупреждений
import warnings
# включим игнорирование предупреждений
warnings.filterwarnings("ignore")

/Users/artemgruzdev/opt/anaconda3/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (SMAPE). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_full_state_property`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


In [2]:
# функция установки начального значения генератора случайных чисел
def set_seed(seed: int = 42):
    # установка генератора случайных чисел python 
    random.seed(seed)
    # установка генератора случайных чисел numpy
    np.random.seed(seed)

In [3]:
def init_logger(config: dict, project: str = "wandb-sweeps", tags: Optional[list] = ["test", "sweeps"]):
    # создание пустого списка для хранения обьектов WandbLogge
    tslogger.loggers = []
    # создание экземпляра WandbLogge. W&B предоставляет облегченную оболочку для 
    # регистрации экспериментов машинного обучения
    wblogger = WandbLogger(project=project, tags=tags, config=config)
    # добавление обьекта WandbLogge в список
    tslogger.add(wblogger)

In [4]:
def dataloader(file_path: Path, freq: str = "D") -> TSDataset:
    df = pd.read_csv(file_path)
    df = TSDataset.to_dataset(df)
    ts = TSDataset(df=df, freq=freq)
    return ts

In [5]:
# целевая функция Optuna.
def objective(trial: optuna.Trial, metric_name: str, ts: TSDataset, horizon: int, lags: int, seed: int):

    # установка начального значения генератора случайных чисел 
    set_seed(seed)

    #  создадим экземпляр Pipeline. Определить модель и характеристики
    pipeline = Pipeline(
        # определим модель
        model=CatBoostModelMultiSegment(
            iterations=trial.suggest_int("iterations", 10, 100),
            depth=trial.suggest_int("depth", 1, 12),
        ),
        # определим список трансформеров
        transforms=[
            StandardScalerTransform("target"),
            SegmentEncoderTransform(),
            LagTransform(in_column="target", 
                         lags=list(range(horizon, horizon + 
                                         trial.suggest_int("lags", 1, lags)))),
        ],
        # установка горизонта для прогнозов
        horizon=horizon,
    )

    # инициализируем WandB logger
    init_logger(pipeline.to_dict())

    # запускаем перекрестную проверку с подсчетом метрик
    metrics, _, _ = pipeline.backtest(ts=ts, metrics=[MAE(), SMAPE(), Sign(), MSE()])
    
    # возвращаем средние значения по найденным метрикам
    return metrics[metric_name].mean()


In [6]:
# функция оптимизации Optuna
def run_optuna(
    # горизонт для прогнозов
    horizon: int = 14,
    # метрика
    metric_name: str = "MAE",
    # URL-адрес базы данных,
    storage: str = "sqlite:///optuna.db",
    # название 
    study_name: Optional[str] = None,
    # количество проходов
    n_trials: int = 5,
    # путь к файлу данных
    file_path: Path = "Data/example_dataset.csv",
    # Направление оптимизации
    direction: str = "minimize",
    # частота временного ряда
    freq: str = "D",
    # задание количества лагов
    lags: int = 24,
    # задание начального значения счетчика случайных значений
    seed: int = 11,
):

    # загрузка данных
    ts = dataloader(file_path, freq=freq)

    # создание сессии оптимизации Optuna
    study = optuna.create_study(
        # URL базы данных. 
        storage=storage,
        study_name=study_name,
        # объект сэмплера
        sampler=optuna.samplers.TPESampler(multivariate=True, group=True),
        # флаг управления поведением при разрешении конфликта
        load_if_exists=True,
        # Направление оптимизации
        direction=direction,
    )

    # запуск оптимизации Optuna
    study.optimize(
        partial(objective, metric_name=metric_name, ts=ts, horizon=horizon, 
                lags=lags, seed=seed), n_trials=n_trials
        )


In [7]:
# Вызов функции run_optuna, запуск оптимизации
run_optuna(file_path = 'Data/example_dataset.csv')

[I 2022-11-24 22:42:03,025] A new study created in RDB with name: no-name-f098894e-2728-4cfe-a399-796ec1dd66f7
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
wandb: Currently logged in as: gewissta. Use `wandb login --relogin` to force relogin


MAE_mean,▁
MAE_median,▁
MAE_percentile_25,▁
MAE_percentile_5,▁
MAE_percentile_75,▁
MAE_percentile_95,▁
MAE_std,▁
MSE_mean,▁
MSE_median,▁
MSE_percentile_25,▁
MSE_percentile_5,▁


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   25.4s remaining:    0.0s


MAE_mean,▁
MAE_median,▁
MAE_percentile_25,▁
MAE_percentile_5,▁
MAE_percentile_75,▁
MAE_percentile_95,▁
MAE_std,▁
MSE_mean,▁
MSE_median,▁
MSE_percentile_25,▁
MSE_percentile_5,▁


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   49.8s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.6min remaining:    0.0s


MAE_mean,▁
MAE_median,▁
MAE_percentile_25,▁
MAE_percentile_5,▁
MAE_percentile_75,▁
MAE_percentile_95,▁
MAE_std,▁
MSE_mean,▁
MSE_median,▁
MSE_percentile_25,▁
MSE_percentile_5,▁


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.0min finished


MAE_mean,▁
MAE_median,▁
MAE_percentile_25,▁
MAE_percentile_5,▁
MAE_percentile_75,▁
MAE_percentile_95,▁
MAE_std,▁
MSE_mean,▁
MSE_median,▁
MSE_percentile_25,▁
MSE_percentile_5,▁


[I 2022-11-24 22:54:01,359] Trial 4 finished with value: 45.47893601699648 and parameters: {'iterations': 62, 'depth': 6, 'lags': 19}. Best is trial 2 with value: 43.89384662813002.
